# 编译与运行环境

C/C++是静态语言,需要编译后才能执行.而编译的工具就是编译器.最常见的C++编译器有:

+ `GNU G++`开源大佬,linux上编译器的的代名词,在osx上也可以使用homebrew安装,windows上则有MinGW作为其移植版.
+ `llvm+Clang`苹果资助的编译器,也是osx下默认的编译器.有不少黑科技,比如C++的repl,jit技术等,一点不像保守的苹果资助的东西.
+ `MSVC`微软家的御用编译器,vs内置的编译器.只能在windows下使用,要在命令行中使用需要将其加入环境变量`Path`


对于C/C++来说,编译其本质就是将源文件转化为二进制文件或者可执行文件的过程,我们以一个小程序来作为例子,使用`gnu g++`工具来演示整个编译过程:

In [1]:
%%writefile ../codes/C1_Cpp_tools/Cpp_compiler/section1/src/helloworld.cpp
#include <stdio.h>

int main(){
    printf("hello world");
    return 0;
}

Writing ../codes/C1_Cpp_tools/Cpp_compiler/section1/src/helloworld.cpp


## 步骤一:解读头文件,结合源文件,组合生成真正的待编译代码
    
这一步可以使用`-E`指令实现.这个过程只激活预处理,不生成文件,而是输出到标准输出.因此你需要把它重定向到一个输出文件里.,一般来说我们用后缀`.i`保存这个文件.

这一步实际上做这些事儿:

+ 宏的替换
+ 注释的消除
+ 还有找到相关的库文件
+ 执行模板,将模板翻译成源码的实现
+ 将#include文件的全部内容插入.若用`<>`括起文件则在系统的INCLUDE目录中寻找文件,若用`" "`括起文件则在当前目录中寻找文件.

In [2]:
!g++ -E ../codes/C1_Cpp_tools/Cpp_compiler/section1/src/helloworld.cpp

# 1 "../codes/C1_Cpp_tools/Cpp_compiler/section1/src/helloworld.cpp"
# 1 "<command-line>"
# 1 "../codes/C1_Cpp_tools/Cpp_compiler/section1/src/helloworld.cpp"
# 1 "c:\\users\\87\\anaconda3\\mingw\\bin\\../lib/gcc/x86_64-w64-mingw32/4.7.0/../../../../x86_64-w64-mingw32/include/stdio.h" 1 3
# 9 "c:\\users\\87\\anaconda3\\mingw\\bin\\../lib/gcc/x86_64-w64-mingw32/4.7.0/../../../../x86_64-w64-mingw32/include/stdio.h" 3
# 1 "c:\\users\\87\\anaconda3\\mingw\\bin\\../lib/gcc/x86_64-w64-mingw32/4.7.0/../../../../x86_64-w64-mingw32/include/_mingw.h" 1 3
# 10 "c:\\users\\87\\anaconda3\\mingw\\bin\\../lib/gcc/x86_64-w64-mingw32/4.7.0/../../../../x86_64-w64-mingw32/include/_mingw.h" 3
# 1 "c:\\users\\87\\anaconda3\\mingw\\bin\\../lib/gcc/x86_64-w64-mingw32/4.7.0/../../../../x86_64-w64-mingw32/include/_mingw_mac.h" 1 3
# 32 "c:\\users\\87\\anaconda3\\mingw\\bin\\../lib/gcc/x86_64-w64-mingw32/4.7.0/../../../../x86_64-w64-mingw32/include/_mingw_mac.h" 3
       
# 41 "c:\\users\\87\\anaconda3\\mingw\\

## 步骤二:将代码转译成汇编语言

这一步可以使用`-S`指令实现.这个过程就是将上一步的这串字符转译成汇编代码.汇编代码说白了就是二进制代码中操作与资源的的替代而已.因此说C/C++和汇编其实也就一步之遥.这也是为什么很多时候嵌入式设备使用C语言编写的原因.


生成`helloworld.s`文件,`.s`文件表示是汇编文件，用编辑器打开就都是汇编指令了


In [3]:
!g++ -o ../codes/C1_Cpp_tools/Cpp_compiler/section1/src/helloworld.s \
-S ../codes/C1_Cpp_tools/Cpp_compiler/section1/src/helloworld.cpp

In [4]:
!cat ../codes/C1_Cpp_tools/Cpp_compiler/section1/src/helloworld.s

'cat' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


## 步骤三:将汇编语言的代码再编译为二进制代码

这一步使用`-c`指令,将汇编代码编译位二进制代码.注意这依然不是可执行文件,因为其中使用的函数很可能并不包含在二进制文件中.

In [5]:
!g++ -o ../codes/C1_Cpp_tools/Cpp_compiler/section1/src/helloworld.o \
-c ../codes/Cpp_tools/Cpp_compiler/section1/src/helloworld.cpp

g++.exe: error: ../codes/Cpp_tools/Cpp_compiler/section1/src/helloworld.cpp: No such file or directory
g++.exe: fatal error: no input files
compilation terminated.


In [6]:
!cat ../codes/C1_Cpp_tools/Cpp_compiler/section1/src/helloworld.o

'cat' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


## 步骤四: 链接相关的二进制代码生成可执行文件

这一步就是将与文件相关的函数库引入以生成最终的可执行文件.

In [7]:
!g++ -o ../codes/C1_Cpp_tools/Cpp_compiler/section1/bin/helloworld \
../codes/C1_Cpp_tools/Cpp_compiler/section1/src/helloworld.o

g++.exe: error: ../codes/C1_Cpp_tools/Cpp_compiler/section1/src/helloworld.o: No such file or directory
g++.exe: fatal error: no input files
compilation terminated.


In [8]:
!../codes/C1_Cpp_tools/Cpp_compiler/section1/bin/helloworld

'..' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


# 多文件编译

在第一helloworld那章已经提过如何编译一个多文件构成的项目,这边以一个稍微复杂些的例子来演示

In [9]:
%%writefile ../codes/C1_Cpp_tools/Cpp_compiler/section2/include/binary_vector.hpp
#ifndef BINARY_VECTOR_H
#define BINARY_VECTOR_H//一般是文件名的大写 头文件结尾写上一行
namespace Vector{
typedef struct {
    float x;
    float y;
} BINARY_VECTOR;
}

#endif

Writing ../codes/C1_Cpp_tools/Cpp_compiler/section2/include/binary_vector.hpp


In [10]:
%%writefile ../codes/C1_Cpp_tools/Cpp_compiler/section2/include/unary_operator.hpp
#ifndef UNARY_OPERATOR_H
#define UNARY_OPERATOR_H//一般是文件名的大写 头文件结尾写上一行
#include "binary_vector.hpp"
#include <math.h>
namespace Vector{
float mod(Vector::BINARY_VECTOR);
}
#endif

Writing ../codes/C1_Cpp_tools/Cpp_compiler/section2/include/unary_operator.hpp


In [11]:
%%writefile ../codes/C1_Cpp_tools/Cpp_compiler/section2/src/unary_operator.cpp
#include "unary_operator.hpp"
float Vector::mod(Vector::BINARY_VECTOR a){
    float result = sqrt(a.x*a.x+a.y*a.y);
    return result;
}

Writing ../codes/C1_Cpp_tools/Cpp_compiler/section2/src/unary_operator.cpp


In [12]:
%%writefile ../codes/C1_Cpp_tools/Cpp_compiler/section2/include/binary_operator.hpp
#ifndef BINARY_OPERATOR_H
#define BINARY_OPERATOR_H//一般是文件名的大写 头文件结尾写上一行
#include "binary_vector.hpp"
namespace Vector{
Vector::BINARY_VECTOR add(Vector::BINARY_VECTOR,Vector::BINARY_VECTOR);
float mul(Vector::BINARY_VECTOR,Vector::BINARY_VECTOR);
}
#endif

Writing ../codes/C1_Cpp_tools/Cpp_compiler/section2/include/binary_operator.hpp


In [13]:
%%writefile ../codes/C1_Cpp_tools/Cpp_compiler/section2/src/binary_operator.cpp
#include "binary_operator.hpp"
Vector::BINARY_VECTOR Vector::add(Vector::BINARY_VECTOR a,Vector::BINARY_VECTOR b){
    Vector::BINARY_VECTOR result = {a.x+b.x,a.y+b.y};
    return result;
}
float Vector::mul(Vector::BINARY_VECTOR a,Vector::BINARY_VECTOR b){
    float result = a.x*b.x+a.y*b.y;
    return result;
}

Writing ../codes/C1_Cpp_tools/Cpp_compiler/section2/src/binary_operator.cpp


In [14]:
%%writefile ../codes/C1_Cpp_tools/Cpp_compiler/section2/test/test.cpp
#include <stdio.h>
#include "binary_vector.hpp"
#include "binary_operator.hpp"
#include "unary_operator.hpp"

int main(void){
    Vector::BINARY_VECTOR A = {1,2},B = {3,4};
    printf("mod(A)=%f\n",Vector::mod(A));
    printf("mod(B)=%f\n",Vector::mod(B));
    printf("mul(A,B)=%f\n",Vector::mul(A,B));
    Vector::BINARY_VECTOR result = Vector::add(A,B);
    printf("add(A,B)=<%f,%f>\n",result.x,result.y);
}


Writing ../codes/C1_Cpp_tools/Cpp_compiler/section2/test/test.cpp


In [15]:
!g++ -o ../codes/C1_Cpp_tools/Cpp_compiler/section2/bin/test \
-I ../codes/C1_Cpp_tools/Cpp_compiler/section2/include \
../codes/C1_Cpp_tools/Cpp_compiler/section2/src/*.cpp \
../codes/C1_Cpp_tools/Cpp_compiler/section2/test/test.cpp

In [16]:
!../codes/C1_Cpp_tools/Cpp_compiler/section2/bin/test

'..' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


## 组织对外的接口

上面的例子比较随性,基本上是一个头文件对应一个实现用的源文件.事实上头文件和实现并没有什么规定要这样对应,只是习惯上会这么做而已.

头文件是用来申明对外接口的工具.其实只要申明的被实现了就可以使用.我们可以参考stl的设计.按照功能为接口分类,用多个头文件描述同一个模块,也可以像很多第三方工具一样使用单一的头文件将接口描述,而在实现上使用多源文件.

本节的代码主要是整理上面的代码,我们知道C++可以用面向对象的范式编程.这也是C++诞生的初衷之一.作为pythoner,我们也更加熟悉使用类来抽象我们的逻辑.我们将上面的代码整理成使用类封装的形式.

In [17]:
%%writefile ../codes/C1_Cpp_tools/Cpp_compiler/section3/include/vector.hpp
#ifndef BINARY_VECTOR_HPP
#define BINARY_VECTOR_HPP//一般是文件名的大写 头文件结尾写上一行
namespace Vector{
class BinaryVector{
public:
    float x;
    float y;
    
    BinaryVector();
    BinaryVector(const BinaryVector&);
    BinaryVector operator=(const BinaryVector &);
    BinaryVector(BinaryVector&&);
    BinaryVector operator=(BinaryVector &&);
    
    BinaryVector(float,float);
    BinaryVector(int,float);
    BinaryVector(int,int);
    BinaryVector(float,int);
    
    
    float mod();
    float mul(const BinaryVector&);
    BinaryVector add(const BinaryVector&);
    
    float operator*(const BinaryVector&);
    BinaryVector operator+(const BinaryVector&);
};
}

#endif

Writing ../codes/C1_Cpp_tools/Cpp_compiler/section3/include/vector.hpp


C/C++的特点之一就是申明与实现分离,这也决定了其开发方式和python截然相反,python一般是想到哪里写到哪里,比较适合敏捷开发,而C/C++明显更适合先整体构思好了再动手写.因此才说python适合原型开发快速迭代,而C/C++适合写稳定的业务.

In [18]:
%%writefile ../codes/C1_Cpp_tools/Cpp_compiler/section3/src/vector.cpp
#include <cmath>
#include "vector.hpp"

Vector::BinaryVector::BinaryVector()
:x(0.0),y(0.0){
}
                            
Vector::BinaryVector::BinaryVector(const Vector::BinaryVector & a)
:x(a.x),y(a.y){
}
    
Vector::BinaryVector Vector::BinaryVector::operator=(const Vector::BinaryVector & a){
    if (this != &a) { 
        x = a.x; 
        y = a.y; 
    } 
    return *this; 
}

Vector::BinaryVector::BinaryVector(Vector::BinaryVector && a)
:x(a.x),y(a.y){
    a.x = 0.0;
    a.y = 0.0;
}
    
Vector::BinaryVector Vector::BinaryVector::operator=(Vector::BinaryVector && a){
    if (this != &a) { 
        x = a.x; 
        y = a.y; 
        a.x = 0.0;
        a.y = 0.0;
    }
    
    return *this; 
}
    
Vector::BinaryVector::BinaryVector(float in_x,float in_y)
:x(in_x),y(in_y){
}
    
Vector::BinaryVector::BinaryVector(int in_x,int in_y)
:x(static_cast<float>(in_x)),y(static_cast<float>(in_y)){
}
    
Vector::BinaryVector::BinaryVector(int in_x,float in_y)
:x(static_cast<float>(in_x)),y(in_y){
}
    
Vector::BinaryVector::BinaryVector(float in_x,int in_y)
:x(in_x),y(static_cast<float>(in_y)){
}
                
float Vector::BinaryVector::mod(){
    float result = sqrt(x*x+y*y);
    return result;
}
float Vector::BinaryVector::mul(const Vector::BinaryVector & that_vector){
    float result = x*that_vector.x+y*that_vector.y;
    return result;
}      
    
Vector::BinaryVector Vector::BinaryVector::add(const Vector::BinaryVector & that_vector){
    Vector::BinaryVector result(x+that_vector.x,y+that_vector.y);
    return result;
    
}
                            
float Vector::BinaryVector::operator*(const Vector::BinaryVector& that_vector){
    return mul(that_vector);
}
Vector::BinaryVector Vector::BinaryVector::operator+(const Vector::BinaryVector& that_vector){
    return add(that_vector);
}

Writing ../codes/C1_Cpp_tools/Cpp_compiler/section3/src/vector.cpp


In [19]:
%%writefile ../codes/C1_Cpp_tools/Cpp_compiler/section3/test/test.cpp
#include <stdio.h>
#include "vector.hpp"

using Vector::BinaryVector;

int main(){
    BinaryVector A(1,2),B(3,4);
    printf("mod(A)=%f\n",A.mod());
    printf("mod(B)=%f\n",B.mod());
    printf("mul(A,B)=%f\n",A*B);
    BinaryVector result = A+B;
    printf("add(A,B)=<%f,%f>\n",result.x,result.y);
}


Writing ../codes/C1_Cpp_tools/Cpp_compiler/section3/test/test.cpp


In [20]:
!g++ -std=c++11 \
-o ../codes/C1_Cpp_tools/Cpp_compiler/section3/bin/test \
-I ../codes/C1_Cpp_tools/Cpp_compiler/section3/include \
../codes/C1_Cpp_tools/Cpp_compiler/section3/src/*.cpp \
../codes/C1_Cpp_tools/Cpp_compiler/section3/test/test.cpp

In [21]:
!../codes/C1_Cpp_tools/Cpp_compiler/section3/bin/test

'..' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


# 链接库

Java当初以跨平台作为重要的卖点,号称一次编译随处运行.其实C/C++当初也是这么宣传的,只不过对于C/C++来说跨平台指的是写好源码,头文件后可以在各个平台编译运行(系统资源接口的调用除外).但是显然这种跨平台很不彻底.而且很关键的一点是C++因为模板,内联函数等特性编译在静态语言中是数一数二的慢.这更加不便于团队合作和复用.

实际上还有另外一种复用的方式,就是将代码编译成链接库.再在步骤四的时候链接到最终的代码中.链接库分为静态链接库和动态链接库两种.

静态链接库与动态链接库都是共享代码的方式

链接库是由第三步的.o文件组合而成,各个系统平台使用不同的规范实现,具体是什么规范这边不细说.总之不同平台编译出来的链接库是不能兼容的.但其实问题也不太大,完全可以编译好后共享出去.

## 链接库的使用方法

无论静态链接库还是动态链接库,命名规则都是`libxxx+后缀`,但编译的时候指定的名字就是`xxx`.而调用方法都是用`#include`导入后就可以直接使用.

很多时候我们为了隐藏代码实现,在共享的时候可以指定好平台操作系统,之后只共享出链接库和头文件.这种方式不仅可以防止代码被人篡改或者侵权,更可以精简接口.

## 静态连接库

在linux中,而静态链接库用`.a`为后缀.windows中以`.lib`为后缀.osx中也以`.a`或`.framework`为后缀.当要使用静态链接库时,连接器会找出程序所需的函数,然后将它们拷贝到执行文件,由于这种拷贝是完整的,所以一旦连接成功,静态程序库也就不再需要了.

### 静态链接库的优点 

+ 代码装载速度快，执行速度略比动态链接库快； 

+ 只需保证在开发者的计算机中有正确的库文件,在以二进制形式发布程序时不需考虑在用户的计算机上的库文件是否存在及版本问题.

### 不足之处

+ 使用静态链接生成的可执行文件体积较大,包含相同的公共代码,造成浪费

### 构建静态链接库

我们还是以之前的二维向量作为例子,一般我们用ar(archive,归档的意思)把多个目标文件集合起来生成静态链接库文件.

## 静态连接库

在linux中,而静态链接库用`.a`为后缀.windows中以`.lib`为后缀.osx中也以`.a`或`.framework`为后缀.当要使用静态链接库时,连接器会找出程序所需的函数,然后将它们拷贝到执行文件,由于这种拷贝是完整的,所以一旦连接成功,静态程序库也就不再需要了.

### 静态链接库的优点 

+ 代码装载速度快，执行速度略比动态链接库快； 

+ 只需保证在开发者的计算机中有正确的库文件,在以二进制形式发布程序时不需考虑在用户的计算机上的库文件是否存在及版本问题.

### 不足之处

+ 使用静态链接生成的可执行文件体积较大,包含相同的公共代码,造成浪费

### 构建静态链接库

我们还是以之前的二维向量作为例子,一般我们用ar(archive,归档的意思)把多个目标文件集合起来生成静态链接库文件.

In [22]:
!g++ -c \
-std=c++11 \
-o ../codes/C1_Cpp_tools/Cpp_compiler/section3/lib/vector.o \
-I ../codes/C1_Cpp_tools/Cpp_compiler/section3/include \
../codes/C1_Cpp_tools/Cpp_compiler/section3/src/vector.cpp

In [23]:
!ar -r ../codes/C1_Cpp_tools/Cpp_compiler/section3/lib/libvector.a \
../codes/C1_Cpp_tools/Cpp_compiler/section3/lib/vector.o

C:\Users\87\Anaconda3\Scripts\ar.bat\..\..\MinGW\bin\ar.exe: creating ../codes/C1_Cpp_tools/Cpp_compiler/section3/lib/libvector.a


### 使用静态链接库

要调用静态库我们依然需要头文件,我们需要一个头文件来描述这个静态库中的接口.我们可以使用之前的头文件`vector.hpp`.要使用静态链接库我们需要用`-L`来指定静态库所在的地址,再使用`-l`来指定静态库的名字.注意静态库的名字.

In [24]:
!g++ -std=c++11 \
-o ../codes/C1_Cpp_tools/Cpp_compiler/section3/bin/test_a \
-I ../codes/C1_Cpp_tools/Cpp_compiler/section3/include \
-L ../codes/C1_Cpp_tools/Cpp_compiler/section3/lib \
-l vector \
../codes/C1_Cpp_tools/Cpp_compiler/section3/test/test.cpp

C:\Users\87\AppData\Local\Temp\ccE95ErS.o:test.cpp:(.text+0xbb): undefined reference to `Vector::BinaryVector::BinaryVector(int, int)'
C:\Users\87\AppData\Local\Temp\ccE95ErS.o:test.cpp:(.text+0xd2): undefined reference to `Vector::BinaryVector::BinaryVector(int, int)'
C:\Users\87\AppData\Local\Temp\ccE95ErS.o:test.cpp:(.text+0xde): undefined reference to `Vector::BinaryVector::mod()'
C:\Users\87\AppData\Local\Temp\ccE95ErS.o:test.cpp:(.text+0x127): undefined reference to `Vector::BinaryVector::mod()'
C:\Users\87\AppData\Local\Temp\ccE95ErS.o:test.cpp:(.text+0x174): undefined reference to `Vector::BinaryVector::operator*(Vector::BinaryVector const&)'
C:\Users\87\AppData\Local\Temp\ccE95ErS.o:test.cpp:(.text+0x1c8): undefined reference to `Vector::BinaryVector::operator+(Vector::BinaryVector const&)'
collect2.exe: error: ld returned 1 exit status


In [25]:
!../codes/C1_Cpp_tools/Cpp_compiler/section3/bin/test_a

'..' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


## 动态连接库

在linux中,动态链接库通常用`.so`为后缀.在windows中,动态链接库通常用`.dll`为后缀.在osx中,动态链接库通常用`.dylib`为后缀.


某个程序在运行中要调用某个动态链接库函数的时候,操作系统首先会查看所有正在运行的程序,看在内存里是否已有此库函数的拷贝了.如果有,则让其共享那一个拷贝;只有没有才链接载入.在程序运行的时候,被调用的动态链接库函数被安置在内存的某个地方,所有调用它的程序将指向这个代码段.因此,这些代码必须使用相对地址,而不是绝对地址.在编译的时候,我们需要告诉编译器,这些对象文件是用来做动态链接库的,所以要用地址不无关代码(Position Independent Code (PIC)).

注意:Linux下进行连接的缺省操作是首先连接动态库,也就是说,如果同时存在静态和动态库,不特别指定的话,将与动态库相连接.


###  动态链接库的优点 

+ 更加节省内存并减少页面交换;

+ 动态链接文件与可执行文件独立,只要输出接口不变(即名称、参数、返回值类型和调用约定不变),更换动态链接文件不会对可执行文件造成任何影响,因而极大地提高了可维护性和可扩展性;

+ 不同编程语言编写的程序只要按照函数调用约定就可以调用同一个动态函数;

+ 适用于大规模的软件开发，使开发过程独立、耦合度小,便于不同开发者和开发组织之间进行开发和测试.

### 不足之处

+ 使用动态链接库的应用程序不是自完备的,它依赖的动态模块也要存在,如果使用载入时动态链接,程序启动时发现动态模块不存在,系统将终止程序并给出错误信息.而使用运行时动态链接,系统不会终止,但由于动态库中的导出函数不可用,程序会加载失败
+ 速度比静态链接慢.当某个模块更新后,如果新模块与旧的模块不兼容,那么那些需要该模块才能运行的软件,统统撕掉.

### 编译动态链接库

动态链接库可以直接使用g++编译.需要使用`-fPIC`指定编译成的是共享库,`-shared`指定连接方式是共享型

In [26]:
!g++ -std=c++11 \
-o ../codes/C1_Cpp_tools/Cpp_compiler/section3/lib/libvector_b.so \
-fPIC \
-shared \
-I ../codes/C1_Cpp_tools/Cpp_compiler/section3/include \
../codes/C1_Cpp_tools/Cpp_compiler/section3/src/vector.cpp

../codes/C1_Cpp_tools/Cpp_compiler/section3/src/vector.cpp:1:0: warning: -fPIC ignored for target (all code is position independent) [enabled by default]


In [27]:
!g++ -std=c++11 \
-o ../codes/C1_Cpp_tools/Cpp_compiler/section3/bin/test_b \
-I ../codes/C1_Cpp_tools/Cpp_compiler/section3/include \
-L ../codes/C1_Cpp_tools/Cpp_compiler/section3/lib \
-l vector_b \
../codes/C1_Cpp_tools/Cpp_compiler/section3/test/test.cpp

c:/users/87/anaconda3/mingw/bin/../lib/gcc/x86_64-w64-mingw32/4.7.0/../../../../x86_64-w64-mingw32/bin/ld.exe: cannot find -lvector_b
collect2.exe: error: ld returned 1 exit status


In [28]:
!../codes/Cpp_tools/Cpp_compiler/section3/bin/test_b

'..' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


## C语言兼容的接口设计

很多时候我们需要考虑对C语言的兼容性,因此需要将C++的接口做个封装.封装为C接口需要使用`extern "C"`.

我们可以用这样的模板来写这个接口头文件:

```cpp
#ifndef BINARY_OPERATOR_H
#define BINARY_OPERATOR_H
#ifdef __cplusplus
extern "C" {
#endif
 
/*...*/
 
#ifdef __cplusplus
}
#endif
#endif
```

> `#ifdef _cplusplus/#endif _cplusplus`

    很明显`#ifdef/#endif`,`#ifndef/#endif`用于条件编译,`#ifdef _cplusplus/#endif _cplusplus`——表示如果定义了宏`_cplusplus`，就执行`#ifdef/#endif`之间的语句，否则就不执行。

    在这里为什么需要`#ifdef _cplusplus/#endif _cplusplus`呢？因为C语言中不支持`extern "C"`声明.


> `extern "C"`

    `extern`关键字是C/C++语言中表明函数和全局变量作用范围(可见性)的关键字,该关键字告诉编译器,其声明的函数和变量可以在本模块或其它模块中使用.
    通常,在模块的头文件中对本模块提供给其它模块引用的函数和全局变量以关键字extern声明.例如,如果模块B欲引用该模块A中定义的全局变量和函数时只需包含模块A的头文件即可.这样,模块B中调用模块A中的函数时,在编译阶段,模块B虽然找不到该函数,但是并不会报错;它会在链接阶段中从模块A编译生成的目标代码中找到此函数.

    与`extern`对应的关键字是`static`,被它修饰的全局变量和函数只能在本模块中使用.因此,一个函数或变量只可能被本模块使用时,其不可能被extern “C”修饰.


    被`extern "C"`修饰的变量和函数是按照C语言方式编译和链接的.


    作为一种面向对象的语言,C++支持函数重载,而C则不支持.函数被C++编译后在符号库中的名字与C语言的不同.例如某个函数的原型为`void foo( int x, int y );`该函数被C编译器编译后在符号库中的名字为`_foo`,而C++编译器则会产生像`_foo_int_int`之类的名字(不同的编译器可能生成的名字不同,但是都采用了相同的机制,生成的新名字称为“mangled name”).


    `_foo_int_int`这样的名字包含了函数名,函数参数数量及类型信息.C++就是靠这种机制来实现函数重载的.同样地，C++中的变量除支持局部变量外,还支持类成员变量和全局变量.用户所编写程序的类成员变量可能与全局变量同名,我们以`"."`来区分.而本质上,编译器在进行编译时,与函数的处理相似,也为类中的变量取了一个独一无二的名字,这个名字与用户程序中同名的全局变量名字不同.

    `extern "C"`指令仅指定编译和连接规约,但不影响语义.例如在函数声明中,指定了`extern "C"`,仍然要遵守C++的类型检测和参数转换规则.
    
要构造C语言也可以使用的连接库,需要用C中的结构,包括`struct`,`function`这些将C++中的内容包装起来,下面就是包装的代码

In [29]:
%%writefile ../codes/C1_Cpp_tools/Cpp_compiler/section4/include/vector.h
#ifndef BINARY_VECTOR_H
#define BINARY_VECTOR_H//一般是文件名的大写 头文件结尾写上一行


#ifdef __cplusplus
extern "C" {
#endif
struct _Vector__BinaryVector;//将BinaryVector对象包装到一个结构体中

typedef struct _Vector__BinaryVector Vector__BinaryVector;
 
Vector__BinaryVector * Vector__BinaryVector_GetInstance(void); //构造结构体实例的函数
void Vector__BinaryVector_ReleaseInstance(Vector__BinaryVector **);//析构结构体实例的函数
//获取字面量
float Vector__BinaryVector_getInstanceX(Vector__BinaryVector *);
float Vector__BinaryVector_getInstanceY(Vector__BinaryVector *); 
//设置字面量
void Vector__BinaryVector_setInstanceX(Vector__BinaryVector *,float);
void Vector__BinaryVector_setInstanceY(Vector__BinaryVector *,float);
void Vector__BinaryVector_setInstance(Vector__BinaryVector *,float,float);

//将实例方法包装为函数
float Vector__BinaryVector_mod(Vector__BinaryVector *);
float Vector__BinaryVector_mul( Vector__BinaryVector * , Vector__BinaryVector *); 
Vector__BinaryVector * Vector__BinaryVector_add(Vector__BinaryVector *, Vector__BinaryVector *);

#ifdef __cplusplus
}
#endif
#endif

Writing ../codes/C1_Cpp_tools/Cpp_compiler/section4/include/vector.h


In [30]:
%%writefile ../codes/C1_Cpp_tools/Cpp_compiler/section4/src/vector_wrapper.cpp
#include <cstdio>
#include <stdlib.h> 
#include "vector.h"    
#include "vector.hpp"
#ifdef __cplusplus    
extern "C" {    
#endif 
struct _Vector__BinaryVector{
    Vector::BinaryVector binary_vector;
};

Vector__BinaryVector * Vector__BinaryVector_GetInstance(void){
    //return new Vector__BinaryVector;
    Vector__BinaryVector * p = (Vector__BinaryVector *)malloc(sizeof(Vector__BinaryVector));
    Vector::BinaryVector v;
    p->binary_vector = v;
    return p;
}

    
void Vector__BinaryVector_ReleaseInstance(Vector__BinaryVector **ppInstance){
    //delete *ppInstance;    
    //*ppInstance = 0;    
    free(*ppInstance);
    *ppInstance = 0;
}

float Vector__BinaryVector_getInstanceX(Vector__BinaryVector *ptr_vector){
    return ptr_vector->binary_vector.x;
}
float Vector__BinaryVector_getInstanceY(Vector__BinaryVector *ptr_vector){
    return ptr_vector->binary_vector.y;
}
void Vector__BinaryVector_setInstanceX(Vector__BinaryVector *ptr_vector,float x){
    ptr_vector->binary_vector.x = x;
}
void Vector__BinaryVector_setInstanceY(Vector__BinaryVector *ptr_vector,float y){
    ptr_vector->binary_vector.y = y;
}
void Vector__BinaryVector_setInstance(Vector__BinaryVector *ptr_vector,float x,float y){
    ptr_vector->binary_vector.x = x;
    ptr_vector->binary_vector.y = y;
}

//将实例方法包装为函数
float Vector__BinaryVector_mod(Vector__BinaryVector * ptr_vector){
    return ptr_vector->binary_vector.mod();
}
    
float Vector__BinaryVector_mul(Vector__BinaryVector * ptr_this_vector, Vector__BinaryVector * ptr_that_vector){
    return ptr_this_vector->binary_vector *  ptr_that_vector->binary_vector;
}
Vector__BinaryVector * Vector__BinaryVector_add(Vector__BinaryVector * ptr_this_vector, Vector__BinaryVector * ptr_that_vector){
    Vector::BinaryVector result_ins = ptr_this_vector->binary_vector +  ptr_that_vector->binary_vector;
    Vector__BinaryVector * result = Vector__BinaryVector_GetInstance(); 
    result->binary_vector = result_ins;
    return result;
}
 
#ifdef __cplusplus    
};    
#endif  


Writing ../codes/C1_Cpp_tools/Cpp_compiler/section4/src/vector_wrapper.cpp


In [31]:
%%writefile ../codes/C1_Cpp_tools/Cpp_compiler/section4/test/test.c
#include <stdio.h>
#include "vector.h"

int main(){
    printf("test4\n");
    Vector__BinaryVector * A = Vector__BinaryVector_GetInstance();
    Vector__BinaryVector * B = Vector__BinaryVector_GetInstance();
    Vector__BinaryVector_setInstance(A,1.0,2.0);
    Vector__BinaryVector_setInstance(B,3.0,4.0);
    printf("mod(A)=%f\n",Vector__BinaryVector_mod(A));
    printf("mod(B)=%f\n",Vector__BinaryVector_mod(B));
    printf("mul(A,B)=%f\n",Vector__BinaryVector_mul(A,B));
    Vector__BinaryVector * result = Vector__BinaryVector_add(A,B);
    printf("add(A,B)=<%f,%f>\n",Vector__BinaryVector_getInstanceX(result),Vector__BinaryVector_getInstanceY(result));
}

Writing ../codes/C1_Cpp_tools/Cpp_compiler/section4/test/test.c


如果我们用使用C语言风格的调用方式用`gcc`来编译,一样可以编译起来.这需要我们先将原始代码和包装代码都分别编译为二进制文件`.o`,之后再将其连接配上入口函数的源文件即可,需要注意的是`gcc`可能会因为C++的错误处理和一些关键字而无法被编译通过,这种时候就需要在编译语句末尾加上`-lstdc++`即引入C++标准库来规避这些错误.

In [32]:
!g++ -std=c++11 \
-c \
-o ../codes/C1_Cpp_tools/Cpp_compiler/section4/lib/vector.o \
-I ../codes/C1_Cpp_tools/Cpp_compiler/section3/include \
../codes/C1_Cpp_tools/Cpp_compiler/section3/src/vector.cpp

In [33]:
!g++ -std=c++11 \
-c \
-o ../codes/C1_Cpp_tools/Cpp_compiler/section4/lib/vector_wrapper.o \
-I ../codes/C1_Cpp_tools/Cpp_compiler/section4/include \
-I ../codes/C1_Cpp_tools/Cpp_compiler/section3/include \
../codes/C1_Cpp_tools/Cpp_compiler/section4/src/vector_wrapper.cpp

In [34]:
!gcc  -o ../codes/C1_Cpp_tools/Cpp_compiler/section4/bin/test \
-I ../codes/C1_Cpp_tools/Cpp_compiler/section4/include \
../codes/C1_Cpp_tools/Cpp_compiler/section4/lib/vector.o \
../codes/C1_Cpp_tools/Cpp_compiler/section4/lib/vector_wrapper.o \
../codes/C1_Cpp_tools/Cpp_compiler/section4/test/test.c \
-lstdc++

In [35]:
!../codes/C1_Cpp_tools/Cpp_compiler/section4/bin/test

'..' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


### 为动态库提供C和C++接口

当然更好的方式是将C++代码编译为动态链接库,之后为C语言和C++分别提供调用用的头文件

In [36]:
!g++ -std=c++11 \
-o ../codes/lib/C1_Cpp_tools/Cpp_compiler/section4/libvector_c.so \
-fPIC \
-shared \
-I ../codes/include/C1_Cpp_tools/Cpp_compiler/section3 \
-I ../codes/include/C1_Cpp_tools/Cpp_compiler/section4 \
../codes/src/C1_Cpp_tools/Cpp_compiler/section3/vector.cpp \
../codes/src/C1_Cpp_tools/Cpp_compiler/section4/vector_wrapper.cpp

g++.exe: error: ../codes/src/C1_Cpp_tools/Cpp_compiler/section3/vector.cpp: No such file or directory
g++.exe: error: ../codes/src/C1_Cpp_tools/Cpp_compiler/section4/vector_wrapper.cpp: No such file or directory
g++.exe: fatal error: no input files
compilation terminated.


In [37]:
!gcc -o ../codes/C1_Cpp_tools/Cpp_compiler/section4/bin/test_b \
-I ../codes/C1_Cpp_tools/Cpp_compiler/include/section4 \
-L ../codes/C1_Cpp_tools/Cpp_compiler/lib/section4/ \
-l vector_c \
../codes/C1_Cpp_tools/Cpp_compiler/section4/test/test.c \
-lstdc++

../codes/C1_Cpp_tools/Cpp_compiler/section4/test/test.c:2:20: fatal error: vector.h: No such file or directory
compilation terminated.


In [38]:
!../codes/C1_Cpp_tools/Cpp_compiler/section4/bin/test_b

'..' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


# 使用宏实现跨平台

我们说过C/C++的跨平台是源码级别的跨平台,但实际上因为操作系统功能的实现有区别,接口也有些区别,因此很多时候我们还是得考虑跨平台问题.其实这个问题python中也有,比如到3.6版本为止windows的`ctrl+c`操作都无法直接用于协程程序的中断.

在C/C++编程时我们可以像上面定义C接口时一样用宏语句判断是否需要特殊化处理.一般情况下会出问题的也就是windows,下面是特殊化处理的宏片段:

```cpp
#ifdef _MSC_VER
    #pragma warning(disable:4996)
    #include <windows.h>
    #include <io.h>
#else
    #include <unistd.h>
    #include <sys/time.h>
    #include <pthread.h>
    #define  CRITICAL_SECTION   pthread_mutex_t
    #define  _vsnprintf         vsnprintf
#endif
```
当然了更加方便的方式是无视windows.


# 总结下g++的常用参数

+ -o 指定输出的文件
+ -E 预编译
+ -S 编译为汇编代码
+ -c 编译位二进制文件
+ -L 选项告诉编译器去哪里找需要的库文件
+ -l -lbvector告诉编译器要链接libbvector库
+ -I 选项告诉去哪个文件夹下找头文件
+ -i 指定使用的头文件
+ -static 链接方式禁止使用动态库。

    优点：程序运行不依赖于其他库

    缺点：文件比较大

+ -shared (-G) 此链接方式尽量使用动态库，为默认选项(共享型)

    优点：生成文件比较小

    缺点：运行时需要系统提供动态库

+ -symbolic 此链接方式将在建立共享目标文件的时候,把引用绑定到全局符号上.
+ -fpic 编译器就生成位置无关目标码.适用于共享库(shared library).通常能产生更快更小的代码,但有平台限制.编译出的代码将通过全局偏移表(Global Offset Table)中的常数地址访存，动态装载器将在程序开始执行时解析GOT表项.
+ -fPIC 编译器就输出位置无关目标码.适用于动态连接(dynamic linking),即使分支需要大范围转移.该选项会针对某些特殊机型做了特殊处理,比如适合动态链接并能避免超出GOT大小限制之类的错误.而Open64仅仅支持不会导致GOT表溢出的PIC编译.

+ ***-lstdc++*** gcc使用的,放在编译代码最后防止因为C++语法而报错

# 使用cmake流程化编译

上面的代码我们直接使用g++来手工编译,这没问题,不过我们也看到了每次编译的命令都比python的helloworld长了.像很多复杂的程序用到很多外部依赖的情况下编译都快赶上写业务逻辑了.

在js中我们用glup来流程化编译,在c/c++中的流程化编译工具就太多了,各自适用于不同的平台有着不同的规范,这边介绍的是[cmake](https://cmake.org/)工具.它是一个高层次的流程化编译工具,关键是他跨平台.

Cmake本质上也不提供流程化编译的功能,他其实是用来生成不同平台make文件的,在linux,mac osx上就是`makefile`,正真编译使用`make`命令,在windows上就是`nmake`当然windows下也可以使用`mingw`来编译,自带的`mingwXX-make`来编译makefile.不过这需要在cmake的时候加上额外参数`-G  "MinGW Makefiles"`指定

## 使用`CMakeLists.txt`配置资源与流程

我们看一个简单的例子来看

### 编写CMakerLists.txt

这个例子很简单,只有一个源文件需要编译.我们定义了三个参数:

+ cmake_minimum_required：指定运行此配置文件所需的 CMake 的最低版本
+ project：参数值是 Demo1，该命令表示项目的名称是 Demo1
+ add_executable： 将名为 main.cc 的源文件编译成一个名称为 Demo 的可执行文件

In [39]:
%%writefile ../codes/C1_Cpp_tools/Cpp_compiler/section5/CMakeLists.txt

cmake_minimum_required (VERSION 2.8)
project (Demo1)
set(CMAKE_RUNTIME_OUTPUT_DIRECTORY ${CMAKE_BINARY_DIR}/bin)
aux_source_directory(src DIR_SRCS)
add_executable(Demo ${DIR_SRCS})

Writing ../codes/C1_Cpp_tools/Cpp_compiler/section5/CMakeLists.txt


In [40]:
%%writefile ../codes/C1_Cpp_tools/Cpp_compiler/section5/src/power.cpp
#include <stdio.h>
#include <stdlib.h>

double power(double base, int exponent)
{
    int result = base;
    int i;

    if (exponent == 0) {
        return 1;
    }
    
    for(i = 1; i < exponent; ++i){
        result = result * base;
    }

    return result;
}

int main(int argc, char *argv[])
{
    if (argc < 3){
        printf("Usage: %s base exponent \n", argv[0]);
        return 1;
    }
    double base = atof(argv[1]);
    int exponent = atoi(argv[2]);
    double result = power(base, exponent);
    printf("%g ^ %d is %g\n", base, exponent, result);
    return 0;
}

Writing ../codes/C1_Cpp_tools/Cpp_compiler/section5/src/power.cpp


In [41]:
!cmake ../codes/C1_Cpp_tools/Cpp_compiler/section5/CMakeLists.txt

-- Building for: Visual Studio 14 2015
-- The C compiler identification is MSVC 19.0.24210.0
-- The CXX compiler identification is MSVC 19.0.24210.0
-- Check for working C compiler: C:/Program Files (x86)/Microsoft Visual Studio 14.0/VC/bin/cl.exe
-- Check for working C compiler: C:/Program Files (x86)/Microsoft Visual Studio 14.0/VC/bin/cl.exe -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working CXX compiler: C:/Program Files (x86)/Microsoft Visual Studio 14.0/VC/bin/cl.exe
-- Check for working CXX compiler: C:/Program Files (x86)/Microsoft Visual Studio 14.0/VC/bin/cl.exe -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Configuring done
-- Generating done
-- Build files have been written to: C:/Users/87/Documents/GitHub/my/TutorialForCpp/ipynbs/codes/C1_Cpp_tools/Cpp_compiler/section5


In [42]:
!make -C ../codes/C1_Cpp_tools/Cpp_compiler/section5

'make' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


## Cmake基本用法

Cmake是指令式的配置方式,基本形式就是`cmake_minimum_required (VERSION 2.8)`这样,`()`外面的是参数字段,里面的是内容.

常用的字段有:

> 元信息设置

+ `cmake_minimum_required`：指定运行此配置文件所需的 CMake 的最低版本

+ `project(<proname>)`：参数值是 Demo1，该命令指定项目的名称

> 编译器设置

+ `set(CMAKE_ARCHIVE_OUTPUT_DIRECTORY ${CMAKE_BINARY_DIR}/Lib)`:设置静态连接文件输出目录
+ `set(CMAKE_LIBRARY_OUTPUT_DIRECTORY ${CMAKE_BINARY_DIR}/Lib)`:设置动态态连接文件输出目录
+ `set(CMAKE_RUNTIME_OUTPUT_DIRECTORY ${CMAKE_BINARY_DIR}/Bin)`:设置可执行文件输出目录
+ `include_directories([AFTER|BEFORE] [SYSTEM] dir1 [dir2 ...])`:设置头文件目录
+ `link_directories(directory1 directory2 ...)`:设置连接库所在位置
+ `link_libraries(library1 <debug | optimized> library2 ...)`:添加需要链接的库文件路径
+ `target_link_libraries(<target> [item1 [item2 [...]]][[debug|optimized|general] <item>] ...)`:设置要链接的库文件的名称
+ `aux_source_directory(<dir> <variable>)`: 指定源文件的所在目录,如`aux_source_directory(. DIR_SRCS)`这样就将所有文件夹下的文件名放到了`DIR_SRCS`变量中.`add_executable(Demo ${DIR_SRCS})`就指定好了所有文件.

+ `add_executable(<target> <sourcefiles...>)`： 将源文件编译成一个可执行文件,第一位是目标可执行文件名,后面则是源文件名.

+ add_compile_options(-std=c++11):指定编译目标使用的额外参数,比如支持c++11

+ target_compile_features(<target> <features...>)指定编译目标使用的额外参数

> 链接编译设置

+ `add_library (<target> [type]<sourcefiles...>)`:将源文件编译成一个链接库.


> 子目录设置

+ `add_subdirectory(math)`:添加子目录,子目录中需要有`CMakeLists.txt`文件,编译时会优先编译子目录.

> 测试设置

可以用`make test`执行测试

+ `enable_testing()`:启动测试
+ `add_test(<测试名> <test_target> <args>)`:添加一个测试,需要先把测试代码编译了
+ `set_tests_properties (test_usage PROPERTIES PASS_REGULAR_EXPRESSION "Usage: .* base exponent")`设定测试的属性


+ 定义一个宏，用来简化测试工作

```cmake
macro (do_test arg1 arg2 result)
  add_test (test_${arg1}_${arg2} Demo ${arg1} ${arg2})
  set_tests_properties (test_${arg1}_${arg2}
    PROPERTIES PASS_REGULAR_EXPRESSION ${result})
endmacro (do_test)
```

> debug测试

```cmake
set(CMAKE_BUILD_TYPE "Debug")
set(CMAKE_CXX_FLAGS_DEBUG "$ENV{CXXFLAGS} -O0 -Wall -g -ggdb")
set(CMAKE_CXX_FLAGS_RELEASE "$ENV{CXXFLAGS} -O3 -Wall")
```


### 使用cmake编译我们的二元向量

更多的用法可以看官方文档.本文不会叙述太多.

In [43]:
%%writefile ../codes/C1_Cpp_tools/Cpp_compiler/section6/CMakeLists.txt

cmake_minimum_required (VERSION 2.8)
project (binary_vector)
set(CMAKE_RUNTIME_OUTPUT_DIRECTORY ${CMAKE_BINARY_DIR}/bin)

aux_source_directory(test DIR_SRCS)
add_subdirectory(src)
add_executable(Demo ${DIR_SRCS})

Writing ../codes/C1_Cpp_tools/Cpp_compiler/section6/CMakeLists.txt


In [44]:
%%writefile ../codes/C1_Cpp_tools/Cpp_compiler/section6/src/CMakeLists.txt

include_directories(../include)
aux_source_directory(. DIR_LIB_SRCS)
set(CMAKE_ARCHIVE_OUTPUT_DIRECTORY ${CMAKE_BINARY_DIR}/lib)
add_library(Vector ${DIR_LIB_SRCS})

Writing ../codes/C1_Cpp_tools/Cpp_compiler/section6/src/CMakeLists.txt


FileNotFoundError: [Errno 2] No such file or directory: '../codes/C1_Cpp_tools/Cpp_compiler/section6/src/CMakeLists.txt'